This script is used to generate five subfiles according to the original LAVA. For each file,
we added two additional columns (used for the following classification tasks) by
transformation from the existed columns, see details in https://docs.google.com/document/d/1VfcjkDbBPYZnMjGG4voIvMYs5dLDiAya-d_8TiJ6CWU/edit


Usage: run the whole notebook, the subfiles will be saved to ./data/lava directory, also can get a sense of data distribution.

In [15]:
import pandas as pd
lava_path = "./data/lava/all_verbs.csv"
alternation = pd.read_csv(lava_path, index_col='verb', dtype='object')

In [16]:
inch = alternation.loc[:, 'inch':'non_inch']
sl = alternation.loc[:, 'sl':'sl_nowith']
there = alternation.loc[:, 'there':'non_there']
dat = alternation.loc[:, 'dat_both':'dat_do']
refl = alternation.loc[:, 'refl_op':'refl_only']

In [19]:
new_add = pd.DataFrame(columns=['inchoative', 'causative'])
inch = pd.concat([inch, new_add])
inch['inchoative'] = inch['inch']
inch.loc[inch['non_inch']!='x', 'causative'] = '1'
inch.loc[inch['non_inch'] =='x', 'causative'] = 'x'

new_inch = inch[['inchoative', 'causative']]
# new_inch.to_csv('./data/lava/inch.csv', index_label='verb')
new_inch.value_counts()

inchoative  causative
x           x            299
0           x             93
1           1             73
0           1             51
dtype: int64

In [21]:
new_add = pd.DataFrame(columns=['preposition', '2object'])
dat = pd.concat([dat, new_add])
dat['preposition'] = dat['dat_both']
dat.loc[(dat['dat_both']=="0") & (dat['dative_to']=='x'), 'preposition'] = 'x'
dat.loc[(dat['dat_both']=="0") & (dat['dative_to']=='1'), 'preposition'] = "1"
dat['2object'] = dat['dat_do']
dat.loc[(dat['dat_both']=="1"), '2object'] = "1"
new_dat = dat[['preposition', '2object']]
# new_dat.to_csv('./data/lava/dative.csv', index_label='verb')
new_dat.value_counts()

preposition  2object
0            0          344
x            0           74
1            1           41
0            1           33
1            0           24
dtype: int64

In [22]:
new_add = pd.DataFrame(columns=['with', 'locative'])
sl = pd.concat([sl, new_add])
sl['with'] = sl['sl_noloc']
sl['locative'] = sl['sl_nowith']
sl.loc[sl['sl']=='1', 'with'] = '1'
sl.loc[sl['sl']=='1', 'locative'] = '1'
new_sl = sl[['with', 'locative']]
# new_sl.to_csv('./data/lava/sl.csv', index_label='verb')
new_sl.value_counts()

with  locative
0     0           185
x     x           173
1     0            72
0     1            57
1     1            29
dtype: int64

In [23]:
new_add = pd.DataFrame(columns=['There', 'No-There'])
there = pd.concat([there, new_add])
there['No-There'] = there['non_there']
there['There'] = there['there']
there.loc[(there['non_there']=='x') & (there['there']!='x'), 'There'] = "0"
there.loc[there['there'] == '1', 'No-There'] = "1"
new_there = there[['There', 'No-There']]
# new_there.to_csv('./data/lava/there.csv', index_label='verb')
new_there.value_counts()

There  No-There
x      x           274
0      1            99
       x            93
1      1            50
dtype: int64

In [24]:
#still not sure whether need to put the refl-op[1,0] in the refl class
new_add = pd.DataFrame(columns=['Refl', 'Non-Refl'])
refl = pd.concat([refl, new_add])
refl['Refl'] = refl['refl_only']
refl['Non-Refl'] = refl['refl_op']
refl.loc[(refl['refl_op']=="1")&(refl['refl_only']=="1"), 'Refl'] = 'x'
refl.loc[(refl['refl_op']=="1")&(refl['refl_only']=="1"), 'Non-Refl'] = 'x'
refl.loc[(refl['refl_op']=="1")&(refl['refl_only']=="0"), 'Refl'] = 'x'
new_refl = refl[['Refl', 'Non-Refl']]
# new_refl.to_csv('./data/lava/refl.csv', index_label='verb')
new_refl.value_counts()

Refl  Non-Refl
0     0           419
1     0            84
x     1            11
      x             2
dtype: int64

In [30]:
# Concatenate all subframes and save in one big frame
verb_frames = pd.concat((new_inch, new_dat, new_sl, new_there, new_refl), axis=1)
verb_frames.to_csv('./data/lava/verb_frames.csv', index_label='verb')